# Training a language model with fastai
- train a language model with curated dataset IMDB using pre-trained model AWD_LSTM


In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from fastai.text.all import *

In [3]:
# define timestamp string for saving models
modifier = "aug13_2021"

# Training a language model
- take a pretrained model and train it some more using the IMDB dataset

In [4]:
%%time
# create path object
path = untar_data(URLs.IMDB)
path.ls()

CPU times: user 27.1 s, sys: 21.2 s, total: 48.3 s
Wall time: 4min


(#7) [Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/tmp_clas'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/README')]

In [5]:
%%time
# create TextDataLoaders object
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])
dls = TextDataLoaders.from_folder(path, valid = 'test', is_lm=True, bs=16)

CPU times: user 34.8 s, sys: 9.84 s, total: 44.6 s
Wall time: 1min 32s


In [6]:
dls.show_batch(max_n=2)

,text,text_
0,"xxbos a light - hearted comedy , xxmaj nothing shows us a world that we sometimes wish to escape to : a world of nothing . xxmaj anything you do n't like , be it a stack of bills , a bad memory , or even hunger can disappear at your wish . xxmaj they approached this movie very well , and with an enjoyable starring duo , there were only a","a light - hearted comedy , xxmaj nothing shows us a world that we sometimes wish to escape to : a world of nothing . xxmaj anything you do n't like , be it a stack of bills , a bad memory , or even hunger can disappear at your wish . xxmaj they approached this movie very well , and with an enjoyable starring duo , there were only a few"
1,"butch , what with their thick legs and arms . xxmaj brilliant . \n\n 3 . xxmaj brilliant - especially the way that neatly ties in with the theme of role reversal between the sexes : so utterly original and mind - blowing . xxmaj ellen behaves like a man , wants sex all the time , while her ex xxmaj patrick wants to talk - like a girl . xxmaj xxunk",", what with their thick legs and arms . xxmaj brilliant . \n\n 3 . xxmaj brilliant - especially the way that neatly ties in with the theme of role reversal between the sexes : so utterly original and mind - blowing . xxmaj ellen behaves like a man , wants sex all the time , while her ex xxmaj patrick wants to talk - like a girl . xxmaj xxunk ."


In [7]:
%%time
# define and train model
learn = language_model_learner(dls,AWD_LSTM,metrics=accuracy).to_fp16()
learn.fine_tune(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.566350,4.347948,0.266946,28:51


epoch,train_loss,valid_loss,accuracy,time
0,4.133654,4.032937,0.296276,31:30


CPU times: user 45min 30s, sys: 15min 5s, total: 1h 35s
Wall time: 1h 36s


In [8]:
# get prediction
# preds = learn.predict('The star is', n_words=20)
learn.predict("what comes next", n_words=20)

'what comes next may have been an insult to animation , corruption , and drug use . This is not as bad'

In [9]:
keep_path = learn.path

In [10]:
# workaround to make path writeable
learn.path = Path('/notebooks/temp')

In [11]:
learn.path

Path('/notebooks/temp')

In [12]:
learn.model_dir

'models'

In [13]:
learn.save('lm_'+modifier)

Path('/notebooks/temp/models/lm_aug13_2021.pth')

In [14]:
# workaround to save encoder - need to do this to later load encoder for classifier
learn.save_encoder('ft_'+modifier)

In [15]:
learn.export('lm_model_'+modifier+'.pkl')

In [16]:
learn.summary()

SequentialRNN (Input shape: 16)
Layer (type)         Output Shape         Param #    Trainable 
                     []                  
LSTM                                                           
LSTM                                                           
LSTM                                                           
RNNDropout                                                     
RNNDropout                                                     
RNNDropout                                                     
____________________________________________________________________________
                     16 x 72 x 60008     
Linear                                    24063208   True      
RNNDropout                                                     
____________________________________________________________________________

Total params: 24,063,208
Total trainable params: 24,063,208
Total non-trainable params: 0

Optimizer used: <function Adam at 0x7ff8af453280>
Loss functio

In [17]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(60008, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(60008, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=60008, bias=True)
    (output_dp): RNNDropout()
  )
)